In [6]:
import pandas as pd
import scipy.signal
from scipy.fftpack import fft
import bokeh
from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import *
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

def get_latest_file(path, *paths):
    fullpath = os.path.join(path, *paths)
    files = glob.iglob('C:\\Users\\grayt\\Desktop\\Test_Data\\*') 
    if not files:                
        return None                     
    latest_file = max(files, key=os.path.getctime)
    _, filename = os.path.split(latest_file)
    return filename

data_file = get_latest_file('C:\\Users\\grayt\\Desktop\\Test_Data\\*')
df = pd.read_csv(('C:\\Users\\grayt\\Desktop\\Test_Data\\'+data_file),
                 usecols=[0,1,2,3], names=['time','x','y','z'], header=None)
df['time'] = pd.to_datetime(df['time'])

df['t(s)'] = (df['time'] - dt.datetime(1970,1,1)).dt.total_seconds() 
df['t(s)'] -= df['t(s)'].min()

N = len(df)
fq= N/df['t(s)'].max()
T = 1/fq
Nf = fq*0.5
N = len(df)
print('Average sample interval =', T,'s')
print('Number of samples = ', N)
print('Sample time = ', df['t(s)'].max(),'s') 
print('Uniform sample time?:', np.allclose(np.diff(df['t(s)']), T))
print('Sampling Frequency (Hz) = ', fq)
print('Nyquist Frequency (Hz) = ', Nf)

# Plot X axis vibration data
p1 = bokeh.plotting.figure(title = 'X axis vibration - '+data_file,
                           height=300, width=1200, x_axis_label='Time', 
                           y_axis_label='mG')
p1.line(df['time'],df['x'], line_color="red") 
p1.xaxis.formatter=DatetimeTickFormatter(
                            minutes=["%M:%S.%3Ns"],
                            seconds=["%H:%M:%S.%3Ns"],
                            milliseconds=["%H:%M:%S.%3Ns"])

xf = np.linspace(0,1/(2*T),N//2)
yf = np.fft.rfft(df['x'])
psd = 2.0/N * np.abs(yf[0:np.int(N/2)])

# Plot X axis fft
p2 = bokeh.plotting.figure(title = 'X axis FFT',
                           x_range = (0,(fq//2)),height=300, width=700, x_axis_label='Frequency (Hz)',
                           y_axis_label='Accel (mG)')
p2.line(xf,psd, line_width=1, line_color="red")

# Plot Y axis vibration data
p3 = bokeh.plotting.figure(title = 'Y axis vibration - '+data_file,
                           height=300, width=1200, x_axis_label='Time', 
                           y_axis_label='mG')
p3.line(df['time'],df['y'], line_color="green") 
p3.xaxis.formatter=DatetimeTickFormatter(
                            minutes=["%M:%S.%3Ns"],
                            seconds=["%H:%M:%S.%3Ns"],
                            milliseconds=["%H:%M:%S.%3Ns"])

xf = np.linspace(0,1/(2*T),N//2)
yf = np.fft.rfft(df['y'])
psd = 2.0/N * np.abs(yf[0:np.int(N/2)])

# Plot Y axis fft
p4 = bokeh.plotting.figure(title = 'Y axis FFT',
                           x_range = (0,(fq//2)),height=300, width=700, x_axis_label='Frequency (Hz)',
                           y_axis_label='Accel (mG)')
p4.line(xf,psd, line_width=1, line_color="green")

# Plot Z axis vibration data
p5 = bokeh.plotting.figure(title = 'Z axis vibration - '+data_file,
                           height=300, width=1200, x_axis_label='Time', 
                           y_axis_label='mG')
p5.line(df['time'],df['z'], line_color="blue") 
p5.xaxis.formatter=DatetimeTickFormatter(
                            minutes=["%M:%S.%3Ns"],
                            seconds=["%H:%M:%S.%3Ns"],
                            milliseconds=["%H:%M:%S.%3Ns"])

xf = np.linspace(0,1/(2*T),N//2)
yf = np.fft.rfft(df['z'])
psd = 2.0/N * np.abs(yf[0:np.int(N/2)])

# Plot Z axis fft
p6 = bokeh.plotting.figure(title = 'Z axis FFT',
                           x_range = (0,(fq//2)),height=300, width=700, x_axis_label='Frequency (Hz)',
                           y_axis_label='Accel (mG)')
p6.line(xf,psd, line_width=1, line_color="blue")
    
output_file('index.html')
bokeh.io.show(bokeh.layouts.column((bokeh.layouts.row(p1,p2)),(bokeh.layouts.row(p3,p4)),
                                   (bokeh.layouts.row(p5,p6))))


Average sample interval = 0.0016917317986096922 s
Number of samples =  43856
Sample time =  74.19258975982666 s
Uniform sample time?: False
Sampling Frequency (Hz) =  591.1102462115007
Nyquist Frequency (Hz) =  295.55512310575034
